In [1]:
import os
import requests
import pendulum
from dotenv import load_dotenv
import logging

In [2]:
SAM_PUBLIC_API_KEY = os.environ.get("SAM_PUBLIC_API_KEY")
previous_date = pendulum.now("utc").subtract(days=1).strftime("%Y%m%d")
base_url = "https://api.sam.gov/opportunities/v2/search"

formatted_request_date = pendulum.parse(previous_date, strict=False).format("MM/DD/YYYY")
api_params = {
    "api_key": SAM_PUBLIC_API_KEY,
    "postedFrom": formatted_request_date,
    "postedTo": formatted_request_date,
    "ptype": "o",
    "limit": 1000,
    "offset": 0,
}

In [5]:

opportunities = []

while True:
    res = requests.get(base_url, params=api_params)
    if res.status_code == 200:
        data = res.json()
        records = data.get("opportunitiesData", [])
        for record in records:
            if record.get("description"):
                res = requests.get(record.get("description"), params={"api_key": api_params['api_key']})
                record['descriptionText'] = res.json()
        opportunities.extend(records)

        if len(records) < api_params["limit"]:
            print(f"Finished with {len(opportunities)} records in memory")
            break

        api_params["offset"] += api_params["limit"]
    elif res.status_code == 429:
        logging.warning("Request limit hit -- try again in 24 hours")
        break
    else:
        raise Exception(f"Request failed with status code {res.status_code}")

Finished with 10 records in memory


In [6]:
descs = [opportunity.get("descriptionText") for opportunity in opportunities]

In [7]:
descs

[{'description': '<p><strong>Notice ID</strong></p>\n\n<p><strong>SPMYM224Q0752</strong></p>\n\n<p><strong>THIS SOLICITATION SPMYM224Q075 AND ANY SUBSEQUENT AMENDMENTS WILL BE POSTED TO THE SAM WEBSITE: SAM.GOV</strong></p>\n\n<p>To be considered for award, if the contractor (awardee) is required to implement NIST SP 800-171, the contractor (awardee) shall have a current assessment as detailed in DFARS 252.204-7019 (I.E., not more than 3 years old unless a lesser time is specified in the solicitation)</p>\n\n<p>**Items deemed to be Commercial Off The Shelf (COTS) are exempt from NIST SP 800-171 requirement.&nbsp; Must identify and provide information to contracting officer on bid during solicitation period**</p>\n\n<p>FOR YOUR BID TO BE DEEMED RESPONSIVE A SIGNED AND COMPLETED SOLICITATION MUST BE SUBMITTED VIA EMAIL OR FAX<u><strong> BY THE OFFER DUE DATE ON THE SOLICITATION (8 MAR 2024 BY 1000 AM PST).</strong></u></p>\n\n<p>If you are able to quote please be sure to:</p>\n\n<p>1.&nb

In [14]:
len(descs)

10

In [ ]:
test_desc = opportunities[0].get("description")
test_desc

In [ ]:
res = requests.get(test_desc, params={"api_key": api_params['api_key']})

In [ ]:
res.json()

In [ ]:
opportunities[0]['descriptionText'] = res.json()

In [8]:
opportunities[0]

{'noticeId': 'e7b71f84b8e1485dbc3c96913bd80050',
 'title': 'BOBBIN / ARRAY PROBE',
 'solicitationNumber': 'SPMYM2244Q0752',
 'fullParentPathName': 'DEPT OF DEFENSE.DEFENSE LOGISTICS AGENCY.DLA MARITIME.DLA MARITIME SHIPYARDS.DLA MARITIME - PUGET SOUND',
 'fullParentPathCode': '097.97AS.DLA MARITIME.DLA MARITIME SHIPYDS.SPMYM2',
 'postedDate': '2024-03-09',
 'type': 'Solicitation',
 'baseType': 'Solicitation',
 'archiveType': 'auto15',
 'archiveDate': '2024-03-23',
 'typeOfSetAsideDescription': 'Total Small Business Set-Aside (FAR 19.5)',
 'typeOfSetAside': 'SBA',
 'responseDeadLine': '2024-03-08T10:00:00-08:00',
 'naicsCode': '334513',
 'naicsCodes': ['334513'],
 'classificationCode': '6635',
 'active': 'Yes',
 'award': None,
 'pointOfContact': [{'fax': '3604764121',
   'type': 'primary',
   'email': 'LINA.1.CRUZ@DLA.MIL',
   'phone': '3608139267',
   'title': None,
   'fullName': 'LINA CRUZ'}],
 'description': 'https://api.sam.gov/prod/opportunities/v1/noticedesc?noticeid=e7b71f84b8e1

In [ ]:
res.content